In [1]:
import sys
import os
sys.path.append(os.path.abspath('../src'))

Importing data for EDA

In [2]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [14]:
from data_preparation import DateTransformer



In [3]:
df_train = pd.read_csv("C:\\Users\\Aman\\Desktop\\kifyaw4\\rossmann-store-sales\\train.csv")
print(df_train.shape)
df_train.head()

C:\Users\Aman\AppData\Local\Temp\ipykernel_14376\3347223601.py:1: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  df_train = pd.read_csv("C:\\Users\\Aman\\Desktop\\kifyaw4\\rossmann-store-sales\\train.csv")


(1017209, 9)


,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,1,5,2015-07-31,5263,555,1,1,0,1
1,2,5,2015-07-31,6064,625,1,1,0,1
2,3,5,2015-07-31,8314,821,1,1,0,1
3,4,5,2015-07-31,13995,1498,1,1,0,1
4,5,5,2015-07-31,4822,559,1,1,0,1


In [ ]:
df_train['StateHoliday'] = df_train['StateHoliday'].astype(str)


In [13]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1017209 entries, 0 to 1017208
Data columns (total 9 columns):
 #   Column         Non-Null Count    Dtype 
---  ------         --------------    ----- 
 0   Store          1017209 non-null  int64 
 1   DayOfWeek      1017209 non-null  int64 
 2   Date           1017209 non-null  object
 3   Sales          1017209 non-null  int64 
 4   Customers      1017209 non-null  int64 
 5   Open           1017209 non-null  int64 
 6   Promo          1017209 non-null  int64 
 7   StateHoliday   1017209 non-null  object
 8   SchoolHoliday  1017209 non-null  int64 
dtypes: int64(7), object(2)
memory usage: 69.8+ MB


In [22]:

num_feature = df_train[df_train.select_dtypes(include='number').columns]
cat_feature = df_train[['StateHoliday']]
date_feature = df_train[['Date']]
    

In [17]:
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer

In [18]:
def create_data_preparation_pipeline(numerical_features, categorical_features, date_column=None):
    # Numerical pipeline
    numerical_pipeline = Pipeline([
        ('imputer', SimpleImputer(strategy='mean')),  # Handle missing values
        ('scaler', StandardScaler())  # Normalize/Standardize
    ])

    # Categorical pipeline
    categorical_pipeline = Pipeline([
        ('imputer', SimpleImputer(strategy='most_frequent')),  # Handle missing values
        ('encoder', OneHotEncoder(handle_unknown='ignore'))  # One-hot encoding
    ])

    # Combine pipelines
    transformers = []
    if numerical_features:
        transformers.append(('num', numerical_pipeline, numerical_features))
    if categorical_features:
        transformers.append(('cat', categorical_pipeline, categorical_features))
    if date_column:
        transformers.append(('date', DateTransformer(date_column), date_column))

    preprocessor = ColumnTransformer(transformers)

    # Full pipeline
    full_pipeline = Pipeline([
        ('preprocessing', preprocessor)  # Preprocessing
    ])

    return full_pipeline


In [23]:
create_data_preparation_pipeline(num_feature, categorical_features=cat_feature, date_column=date_feature)

ValueError: The truth value of a DataFrame is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().